In [1]:
pip install selenium pandas

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.4 MB 1.5 MB/s eta 0:00:06
   ---- ----------------------------------- 1.0/9.4 MB 1.6 MB/s eta 0:00:06
   ----- ---------------------------------- 1.3/9.4 MB 1.6 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/9.4 MB 1.6 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/9.4 MB 1.6 MB/s eta 0:00:05
   ---------- ----------------------------- 2.4/9.4 MB 1.6 MB/s eta 0:00:05
   ----------- ---------------------------- 2.6/9.4 MB 1.6 MB/s eta 0:00:05
   ------------- -------------------------- 3.1/9.4 MB 1.5 MB/s eta 0:00:05
   -------------- ------------------------- 3.4/9.4 MB 1.6 MB/s eta 0:00:04
   --------------- ------------------------ 3.7/9.4 MB 1.6 MB/s eta 0:00:04
   ----------------- -----------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

In [ ]:
# === 1. CONFIGURATION SELENIUM ===
CHROMEDRIVER_PATH = r"chrome/chromedriver.exe" 
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--lang=en-US")
options.add_argument("accept-language=en-US,en;q=0.9")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)

# === 2. SEARCH SETTING ===
subreddits = ["depression", "Anxiety", "SuicideWatch"]
keywords = [
    "death", "anxiety", "suicide", "sad", "broken", "dark", "lost", "exhausted", "panic", "lonely", "die",
    "numb", "worthless", "tired", "hopeless", "drained", "pointless", "empty",
    "overwhelmed", "shaking", "scared", "nervous", "worried", "spiraling",
    "alone", "ignored", "invisible", "unloved", "abandoned", "friendless", "unnoticed",
    "cut", "kill", "gone", "disappear", "hurt", "over", "end", "burnt", "overload", "blank", "shutdown", "collapse", "fatigued", "done", "mute"
]


results = []

for subreddit in subreddits:
    for keyword in keywords:
        query = keyword.replace(" ", "%20")
        url = f"https://www.reddit.com/r/{subreddit}/search/?q={query}&restrict_sr=1&sort=new"
        print(f"{subreddit} → {keyword}")
        driver.get(url)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[data-testid="post-title"]'))
            )

            links = driver.find_elements(By.CSS_SELECTOR, 'a[data-testid="post-title"]')

            for link in links:
                title = link.get_attribute("aria-label") or link.text
                if keyword.lower() in title.lower():
                    href = link.get_attribute("href")
                    results.append({
                        "subreddit": subreddit,
                        "keyword": keyword,
                        "title": title,
                        "url": href
                    })

        except Exception as e:
            print(f"Erreur scraping : {e}")

        time.sleep(2)

driver.quit()

# Export
df = pd.DataFrame(results)
df.to_csv("reddit_filtered_results.csv", index=False, encoding="utf-8")
print(f"\n{len(df)} Post saved.")

depression → death
Erreur scraping : Message: 

depression → anxiety
depression → suicide
depression → sad
depression → borken
depression → dark
depression → lost
depression → exhausted
depression → panic
depression → lonely
depression → die
Anxiety → death
Anxiety → anxiety
Anxiety → suicide
Anxiety → sad
Anxiety → borken
Erreur scraping : Message: 

Anxiety → dark
Anxiety → lost
Anxiety → exhausted
Anxiety → panic
Anxiety → lonely
Anxiety → die
SuicideWatch → death
SuicideWatch → anxiety
SuicideWatch → suicide
SuicideWatch → sad
SuicideWatch → borken
SuicideWatch → dark
SuicideWatch → lost
SuicideWatch → exhausted
SuicideWatch → panic
SuicideWatch → lonely
SuicideWatch → die

77 posts contenant les mots-clés exacts ont été enregistrés.
